In [1]:
import os

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import dask.config
import distributed

# scipy basics
import numpy as np
import rasterio
import rasterio.features
import rasterio.transform
import rasterio.windows

from numba import jit
import concurrent.futures

import boto3
import time
import math
import ctypes

<font size="6">Making cloud and local clusters</font> 

In [2]:
coiled_cluster = coiled.Cluster(
    n_workers=20,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="20 minutes",
    region="us-east-1",
    name="next_gen_forest_carbon_flux_model", 
    # environ={"distributed.nanny.environ": {"MALLOC_TRIM_THRESHOLD_": "1"}},
    account='jterry64'   # Necessary to use the AWS environment that Justin set up in Coiled
)

Output()

Package - bcrypt, bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge

╭──────────────────────────────────── Not Synced with Cluster ─────────────────────────────────────╮
│            ╷                                                                         ╷           │
│   Package  │ Error                                                                   │ Risk      │
│ ╶──────────┼─────────────────────────────────────────────────────────────────────────┼─────────╴ │
│   bcrypt   │ bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge      │ Warning   │
│            ╵                                                                         ╵           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [3]:
# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

<Client: 'tls://10.0.18.21:8786' processes=2 threads=8, memory=29.63 GiB>

In [ ]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

In [ ]:
local_client = Client()
local_client

In [ ]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

<font size="6">Shutting down cloud and local clusters</font> 

In [ ]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()

<font size="6">Analysis</font> 

<font size="4">Paths and functions</font>

In [4]:
# General paths and constants

general_uri = 's3://gfw2-data/forest_change/GLAD_Europe_height_data/'

s3_out_dir = 'climate/European_height_carbon_model/outputs'

def timestr():
    return time.strftime("%Y%m%d_%H_%M_%S")

In [5]:
# Returns list of all chunk boundaries within a bounding box for chunks of a given size
def get_chunk_bounds(chunk_params):

    min_x = chunk_params[0]
    min_y = chunk_params[1]
    max_x = chunk_params[2]
    max_y = chunk_params[3]
    chunk_size = chunk_params[4]
    
    x, y = (min_x, min_y)
    chunks = []

    # Polygon Size
    while y < max_y:
        while x < max_x:
            bounds = [
                x,
                y,
                x + chunk_size,
                y + chunk_size,
            ]
            chunks.append(bounds)
            x += chunk_size
        x = min_x
        y += chunk_size

    return chunks

# Returns the encompassing tile_id string in the form YYN/S_XXXE/W based on a coordinate
def xy_to_tile_id(top_left_x, top_left_y):

    lat_ceil = math.ceil(top_left_y/10.0) * 10
    lng_floor = math.floor(top_left_x/10.0) * 10
    
    lng: str = f"{str(lng_floor).zfill(3)}E" if (lng_floor >= 0) else f"{str(-lng_floor).zfill(3)}W"
    lat: str = f"{str(lat_ceil).zfill(2)}N" if (lat_ceil >= 0) else f"{str(-lat_ceil).zfill(2)}S"

    return f"{lat}_{lng}"

In [6]:
# Lazily opens tile within provided bounds (one chunk)
def get_tile_dataset_rio(uri, bounds, chunk_length):
    try:
        with rasterio.open(uri) as ds:
            return ds.read(1, window=rasterio.windows.from_bounds(*bounds, ds.transform))
    except rasterio.errors.RasterioIOError as e:
        return np.zeros((chunk_length, chunk_length))

<font size="4">Model steps</font>

In [7]:
# Opeates on pixels in each chunk for a given year: forest state, emission factor, removal factor, emissions, removals, carbon stocks
@jit(nopython=True)
def classify(forest_height_previous_block, forest_height_current_block, forest_loss_detection_block, driver_block, planted_forest_type_block, peat_block, 
             land_cover_previous_block, land_cover_next_block, burned_area_two_before_block, burned_area_one_before_block,
             r_s_ratio_block):

    # Output chunks
    forest_states = np.zeros(forest_height_previous_block.shape)
    biomass_C_emission_fraction = np.zeros(forest_height_previous_block.shape)
    dead_C_emission_fraction = np.zeros(forest_height_previous_block.shape)
    soil_C_emission_fraction = np.zeros(forest_height_previous_block.shape)
    agc_removal_factor = np.zeros(forest_height_previous_block.shape)
    bgc_removal_factor = np.zeros(forest_height_previous_block.shape)
    
    # Iterates through all pixels in the chunk
    for row in range(forest_height_previous_block.shape[0]):
        for col in range(forest_height_previous_block.shape[1]):

            # Pixel for each input
            forest_height_previous = forest_height_previous_block[row, col]
            forest_height_current = forest_height_current_block[row, col]
            forest_loss_detection = forest_loss_detection_block[row, col]
            driver = driver_block[row, col]
            planted_forest_type = planted_forest_type_block[row, col]
            peat = peat_block[row, col]
                       
            LC_previous = land_cover_previous_block[row, col]
            LC_next = land_cover_next_block[row, col]
            burned_area_two_before = burned_area_two_before_block[row, col]
            burned_area_one_before = burned_area_one_before_block[row, col]

            r_s_ratio = r_s_ratio_block[row, col]

            # Various definitions
            grassland_forest_previous = (((LC_previous >= 2) & (LC_previous <= 48)) | ((LC_previous >= 102) & (LC_previous <= 148)))
            grassland_forest_next = (((LC_next >= 2) & (LC_next <= 48)) | ((LC_next >= 102) & (LC_next <= 148)))  
            cropland_previous = (LC_previous == 244)
            cropland_next = (LC_next == 244)
            forestry = (driver == 3)
            non_sdpt_forestry = (forestry & (grassland_forest_previous | grassland_forest_next) & (cropland_previous == 0) & (cropland_next == 0))

            burned_area_recent = ((burned_area_two_before != 0) or (burned_area_one_before != 0))

            # The decision tree that produces all the outputs for a pixel for a given year
            if forest_height_previous >= 5 and forest_height_current >= 5:                                     # maintained
                rf = 5
                forest_states[row, col] = 1
                agc_removal_factor[row, col] = rf
                bgc_removal_factor[row, col] = rf * r_s_ratio
            elif forest_height_previous < 5 and forest_height_current >= 5:                                    # gain
                rf = 10
                forest_states[row, col] = 2
                agc_removal_factor[row, col] = rf
                bgc_removal_factor[row, col] = rf * r_s_ratio
            elif ((forest_height_previous >= 5 and forest_height_current < 5) or forest_loss_detection == 1):  # loss
                if planted_forest_type == 0:                                                                   # loss:no SDPT                
                    if non_sdpt_forestry == 0:                                                                 # loss:no SDPT:no non-SDPT forestry                    
                        forest_states[row, col] = 311
                        biomass_C_emission_fraction[row, col] = 0.9
                        dead_C_emission_fraction[row, col] = 0.3
                        soil_C_emission_fraction[row, col] = 0.1
                    else:                                                                                      # loss:no SDPT:non-SDPT forestry         
                        forest_states[row, col] = 312
                        biomass_C_emission_fraction[row, col] = 0.7
                        dead_C_emission_fraction[row, col] = 0.1
                        soil_C_emission_fraction[row, col] = 0.6
                else:                                                                                          # loss:SDPT
                    if burned_area_recent == 0:                                                                # loss:SDPT:not burned recent
                        if peat == 0:                                                                          # loss:SDPT:not burned recent:not peat
                            forest_states[row, col] = 3211
                            biomass_C_emission_fraction[row, col] = 0.6
                            dead_C_emission_fraction[row, col] = 0.2
                            soil_C_emission_fraction[row, col] = 0.25
                        else:                                                                                  # loss:SDPT:not burned recent:peat
                            forest_states[row, col] = 3212
                            biomass_C_emission_fraction[row, col] = 0.75
                            dead_C_emission_fraction[row, col] = 0.1
                            soil_C_emission_fraction[row, col] = 0.25        
                    else:                                                                                      # loss:SDPT:burned recent
                        if peat == 0:                                                                          # loss:SDPT:burned recent:not peat
                            forest_states[row, col] = 3221
                            biomass_C_emission_fraction[row, col] = 0.65
                            dead_C_emission_fraction[row, col] = 0.3
                            soil_C_emission_fraction[row, col] = 0.9 
                        else:                                                                                  # loss:SDPT:burned recent:peat
                            forest_states[row, col] = 3222
                            biomass_C_emission_fraction[row, col] = 0.9
                            dead_C_emission_fraction[row, col] = 0.4
                            soil_C_emission_fraction[row, col] = 0.8 
            else:                                                                                              # no forest
                forest_states[row, col] = 0
                biomass_C_emission_fraction[row, col] = 0
                dead_C_emission_fraction[row, col] = 0
                soil_C_emission_fraction[row, col] = 0
                agc_removal_factor[row, col] = 0
                bgc_removal_factor[row, col] = 0

    return forest_states, biomass_C_emission_fraction, dead_C_emission_fraction, soil_C_emission_fraction, agc_removal_factor, bgc_removal_factor

In [13]:
def process_chunk(bounds, chunk_length_deg, year):
 
    futures = {}
    layers = {}

    bounds_str = "_".join([str(round(x)) for x in bounds])
    chunk_length_pixels = int(chunk_length_deg * 40000/10)

    try:
        
        # submit requests to S3 for layers
        with concurrent.futures.ThreadPoolExecutor() as executor:
            tile_id = xy_to_tile_id(bounds[0], bounds[3])

            dask_print(f"Submitting input retrievals for {bounds_str} in {tile_id} for {year}: {timestr()}")
            
            drivers_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/{tile_id}_tree_cover_loss_driver_processed.tif"
            planted_forest_type_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/planted_forest_type/SDPT_v1/standard/20200730/{tile_id}_plantation_type_oilpalm_woodfiber_other_unmasked.tif"
            peat_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif"
            
            r_s_ratio_uri = f"s3://gfw2-data/climate/carbon_model/BGB_AGB_ratio/processed/20230216/{tile_id}_BGB_AGB_ratio.tif"
   
            forest_height_previous_uri = f'{general_uri}202307_revision/FH_{year-1}.tif'
            forest_height_current_uri = f'{general_uri}202307_revision/FH_{year}.tif'
            forest_loss_detection_uri = f'{general_uri}202307_revision/DFL_{year}.tif'

            preceding_land_cover_uri = f's3://gfw2-data/landcover/composite/2015/{tile_id}_composite_landcover_2015.tif'
            next_land_cover_uri = f's3://gfw2-data/landcover/composite/2020/{tile_id}_composite_landcover_2020.tif'

            burned_area_two_before_uri = f's3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year-2}_50N_010E.tif'
            burned_area_one_before_uri = f's3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year-1}_50N_010E.tif'

            # save as dictionary from future -> name so we can know what layer it is on completion
            futures[executor.submit(get_tile_dataset_rio, drivers_uri, bounds, chunk_length_pixels)] = "drivers"
            futures[executor.submit(get_tile_dataset_rio, planted_forest_type_uri, bounds, chunk_length_pixels)] = "planted_forest_type"
            futures[executor.submit(get_tile_dataset_rio, peat_uri, bounds, chunk_length_pixels)] = "peat"
            
            futures[executor.submit(get_tile_dataset_rio, r_s_ratio_uri, bounds, chunk_length_pixels)] = "r_s_ratio"

            futures[executor.submit(get_tile_dataset_rio, forest_height_previous_uri, bounds, chunk_length_pixels)] = f"forest_height_{year-1}"
            futures[executor.submit(get_tile_dataset_rio, forest_height_current_uri, bounds, chunk_length_pixels)] = f"forest_height_{year}"
            futures[executor.submit(get_tile_dataset_rio, forest_loss_detection_uri, bounds, chunk_length_pixels)] = f"forest_loss_detection_{year}"
            
            futures[executor.submit(get_tile_dataset_rio, preceding_land_cover_uri, bounds, chunk_length_pixels)] = f"preceding_land_cover_for_{year}"
            futures[executor.submit(get_tile_dataset_rio, next_land_cover_uri, bounds, chunk_length_pixels)] = f"next_land_cover_for_{year}"

            futures[executor.submit(get_tile_dataset_rio, burned_area_two_before_uri, bounds, chunk_length_pixels)] = f"burned_area_{year-2}"
            futures[executor.submit(get_tile_dataset_rio, burned_area_one_before_uri, bounds, chunk_length_pixels)] = f"burned_area_{year-1}"

        dask_print(f"Downloading inputs for {bounds_str} in {tile_id} for {year}: {timestr()}")
        
        # wait for requests to come back with data from S3
        for future in concurrent.futures.as_completed(futures):
            layer = futures[future]
            layers[layer] = future.result()
            
        dask_print(f"Classifying {bounds_str} in {tile_id} for {year}: {timestr()}")
        
        # dask_print(agc_current)
        forest_states, biomass_C_emission_fraction, dead_C_emission_fraction, soil_C_emission_fraction, agc_removal_factor, bgc_removal_factor = classify(
            layers[f"forest_height_{year - 1}"], 
            layers[f"forest_height_{year}"], 
            layers[f"forest_loss_detection_{year}"], 
            layers["drivers"],
            layers["planted_forest_type"],
            layers["peat"],
        
            layers[f"preceding_land_cover_for_{year}"],
            layers[f"next_land_cover_for_{year}"],
            layers[f"burned_area_{year-2}"],
            layers[f"burned_area_{year-1}"],

            layers["r_s_ratio"]
        )

        # dask_print(agc_current)
        
        transform = rasterio.transform.from_bounds(*bounds, width=chunk_length_pixels, height=chunk_length_pixels)

        file_info = f'{tile_id}__{year}__{bounds_str}'

        output_dict = {
                "forest_states": [forest_states, "uint16"], 
                "biomass_C_emission_fraction": [biomass_C_emission_fraction, "float32"],
                "dead_C_emission_fraction": [dead_C_emission_fraction, "float32"],
                "soil_C_emission_fraction": [soil_C_emission_fraction, "float32"],
                "agc_removal_factor": [agc_removal_factor, "float32"],
                "bgc_removal_factor": [bgc_removal_factor, "float32"]
               }

        s3_client = boto3.client("s3")

        dask_print(f"Saving {bounds_str} in {tile_id} for {year}: {timestr()}")

        for key, value in output_dict.items():

            file_name = f"{key}__{file_info}__{timestr()}"

            if value[1] == "float32":
                with rasterio.open(f"/tmp/{file_name}.tif", 'w', driver='GTiff', width=chunk_length_pixels, height=chunk_length_pixels, count=1, dtype='float32', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst:
                    dst.write(value[0].astype(rasterio.float32), 1)
            else:
                with rasterio.open(f"/tmp/{file_name}.tif", 'w', driver='GTiff', width=chunk_length_pixels, height=chunk_length_pixels, count=1, dtype='uint16', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst:
                    dst.write(value[0].astype(rasterio.uint16), 1)

            s3_client.upload_file(f"/tmp/{file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{file_name}.tif")
        
        # clear memory of unneeded arrays
        del layers["drivers"]
        del layers["planted_forest_type"]
        del layers["peat"]
        del layers["r_s_ratio"]
        
        del layers[f"preceding_land_cover_for_{year}"]
        del layers[f"next_land_cover_for_{year}"]
        
        del layers[f"forest_height_{year - 1}"]
        del layers[f"forest_height_{year}"]
        del layers[f"forest_loss_detection_{year}"]
        del layers[f"burned_area_{year-2}"]
        del layers[f"burned_area_{year-1}"]

        del forest_states
        del biomass_C_emission_fraction
        del dead_C_emission_fraction
        del soil_C_emission_fraction
        del agc_removal_factor
        del bgc_removal_factor
    
    except Exception as e:
        return f"error: {e}"

    return f"success for {bounds_str} in {year}: {timestr()}"

In [ ]:
%%time

# start year represents the "current year", i.e. the end year of the year pair
start_year = 2002   # full run
# start_year = 2012  # last few years
# start_year = 2021  # final year

years = list(range(start_year, 2022))
print(f"Years to iterate through: {years}")

# chunk_params arguments: W, S, E, N, chunk size (degrees)
chunk_params = [-12, 34, 32, 72, 1]  # all of Europe
# chunk_params = [10, 40, 20, 50, 1]    # 10x10 deg (50N_010E), 100 chunks    -- currently works
# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks   -- currently crashes
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk

chunks = get_chunk_bounds(chunk_params)  # even smaller area that's part of 50N_010E. Final argument is chunk size in degrees.
print(f"Processing {len(chunks)*len(years)} tasks: {len(chunks)} chunks by {len(years)} years")

delayed = [dask.delayed(process_chunk)(chunk, chunk_params[4], year) for chunk in chunks for year in years]
# delayed = [dask.delayed(process_chunk)(chunk, chunk_params[4], start_year) for chunk in chunks]

results = dask.compute(*delayed)
results

In [ ]:
# To run without dask at all
process_chunk([10, 49, 11, 50], 1, start_year)

In [11]:
coiled_client.restart() 

<Client: 'tls://10.0.18.21:8786' processes=19 threads=76, memory=281.19 GiB>

In [ ]:
client.cancel(future) # per https://github.com/dask/distributed/issues/3898#issuecomment-645590511

In [ ]:
# Manually trim memory, per https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

coiled_client.run(trim_memory)

In [ ]:
# aws s3 cp s3://gfw2-data/climate/European_height_carbon_model/outputs/ . --recursive --exclude "*" --include "*10_49_11_50*"

In [ ]:
# Tests rasterio outside of anything else
uri = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif"
bounds = [10, 48, 12, 50]
get_tile_dataset_rio(uri, bounds) 

In [ ]:
# Tests rasterio with concurrent futures but not with dask
uri = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif"
bounds = [10, 48, 12, 50]
futures = {}
layers = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures[executor.submit(get_tile_dataset_rio, uri, bounds)] = "drivers"

for future in concurrent.futures.as_completed(futures):
            layer = futures[future]
            layers[layer] = future.result()

layers["drivers"]